In [2]:
import yaml
import sys
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mave_calibration.main import singleFit, prep_data
import scipy.stats as spsz
from utils.fit_fig import fit_fig
from pathlib import Path
from collections import defaultdict
from mave_calibration.em_opt.constraints import density_constraint_violated
from mave_calibration.em_opt.utils import get_sample_weights,constrained_em_iteration,em_iteration,get_likelihood
import logging
from tqdm.autonotebook import tqdm,trange
from sklearn.cluster import KMeans
from mave_calibration.skew_normal.fit import fit_skew_normal


In [7]:
def fix_to_satisfy_density_constraint(component_parameters, xlims):
    n_components = len(component_parameters)
    rep_failed = False
    for compI, compJ in zip(range(0,n_components-1),range(1,n_components)):
        if rep_failed:
            break
        for _ in range(300):
            if not density_constraint_violated(
                component_parameters[compI], component_parameters[compJ], xlims
            ):
                break
            component_parameters[compI] = [component_parameters[compI][0] - .05 * abs(component_parameters[compI][0]),
                                           component_parameters[compI][1],
                                           component_parameters[compI][2]]
            component_parameters[compJ] = [component_parameters[compJ][0] + .05 * abs(component_parameters[compJ][0]),
                                           component_parameters[compJ][1],
                                           component_parameters[compJ][2]]

        if density_constraint_violated(
            component_parameters[compI], component_parameters[compJ], xlims
        ):
            rep_failed = True
            break
    if rep_failed:
        return [[] for _ in range(n_components)]
    assert not density_constraint_violated(
        component_parameters[0], component_parameters[1], xlims
    )
    return component_parameters

def single_kmeans_init(X, **kwargs):
    """
    Initialize the parameters of the skew normal mixture model using kmeans and the method of moments

    Arguments:
    X: np.array (N,): observed instances

    Optional Keyword Arguments:
    - n_clusters: int: number of clusters to use in kmeans. Default: 2
    - kmeans_init: str: initialization method for kmeans. Options: ["random", "k-means++"]. Default: "random"
    - skewnorm_init_method: str: method to use for fitting the skew normal distribution. Options: ["mle", "mm"]. Default: "mle"
    """
    repeat = 0
    while repeat < 1000:
        n_clusters = kwargs.get("n_clusters", 2)
        init = kwargs.get("kmeans_init", "random")
        kmeans = KMeans(n_clusters=n_clusters, init=init)

        X = np.array(X).reshape((-1, 1))
        kmeans.fit(X)
        cluster_assignments = kmeans.predict(X)

        component_parameters = []
        for i in range(n_clusters):
            X_cluster = X[cluster_assignments == i]
            loc,scale = sps.norm.fit(X_cluster)
            a = np.random.uniform(-.25,.25)
            component_parameters.append((a,float(loc),float(scale)))
        component_parameters = fix_to_satisfy_density_constraint(component_parameters,(X.min(),X.max()))
        if not len(component_parameters[0]):
            repeat += 1
        else:
            return component_parameters, kmeans
    raise ValueError("Failed to initialize")

def random_init(X, **kwargs):
    """
    Initialize the parameters of the skew normal mixture model using random initialization

    Arguments:
    X: np.array (N,): observed instances

    Optional Keyword Arguments:
    - n_clusters: int: number of clusters to use in kmeans. Default: 2
    - skewnorm_init_method: str: method to use for fitting the skew normal distribution. Options: ["mle", "mm"]. Default: "mle"
    """
    repeat = 0
    while repeat < 1000:
        n_clusters = kwargs.get("n_clusters", 2)
        component_parameters = []
        for i in range(n_clusters):
            loc = np.random.uniform(X.min(), X.max())
            scale = np.random.uniform(1e-5, X.max() - X.min())
            a = np.random.uniform(kwargs.get('skew_min',-5),kwargs.get('skew_max',5))
            component_parameters.append((a,loc,scale))
        component_parameters = fix_to_satisfy_density_constraint(component_parameters,(X.min(),X.max()))
        if not len(component_parameters[0]):
            repeat += 1
        else:
            return component_parameters
    raise ValueError("Failed to initialize")

def single_fit(observations,sample_indicators,**kwargs):
    CONSTRAINED=kwargs.get("Constrained",True)
    MAX_N_ITERS = kwargs.get("max_iters", 10000)
    verbose = kwargs.get("verbose",True)
    xlims= (observations.min(),observations.max())
    N_samples = sample_indicators.shape[1]
    N_components = 2
    W = np.ones((N_samples, N_components)) / N_components
    try:
        initial_params,kmeans = single_kmeans_init(observations, n_clusters=N_components)
    except ValueError:
        logging.warning("Failed to initialize")
        return dict(component_params=[[] for _ in range(N_components)],
                    weights=W,
                    likelihoods=[-1 * np.inf])
    W = get_sample_weights(observations, sample_indicators, initial_params, W)
    history = [dict(component_parameters=initial_params, weights=W)]
    # initial likelihood
    likelihoods = np.array(
        [
            get_likelihood(observations, sample_indicators, initial_params, W) / len(sample_indicators),
        ]
    )
    # Check for bad initialization
    try:
        updated_component_params, updated_weights = (
            constrained_em_iteration(observations, sample_indicators, initial_params, W, xlims, iterNum=0)
        )
    except ZeroDivisionError:
        logging.warning("ZeroDivisionError")
        return dict(component_params=initial_params, weights=W, likelihoods=[*likelihoods, -1 * np.inf],kmeans=kmeans)
    likelihoods = np.array(
        [
            *likelihoods,
            get_likelihood(observations, sample_indicators, updated_component_params, updated_weights)
            / len(sample_indicators),
        ]
    )
    # Run the EM algorithm
    if verbose:
        pbar = tqdm(total=MAX_N_ITERS,leave=False,desc="EM Iteration")
    
    for i in range(MAX_N_ITERS):
        history.append(dict(component_parameters=updated_component_params, weights=updated_weights))
        if np.isnan(likelihoods).any():
            raise ValueError()
        if np.isnan(np.concatenate(updated_component_params)).any():
            raise ValueError()
        if np.isnan(updated_weights).any():
            raise ValueError()
        if np.isnan(np.concatenate(updated_component_params)).any():
            raise ValueError(f"NaN in updated component params at iteration {i}\n{updated_component_params}")
        if np.isnan(updated_weights).any():
            raise ValueError(f"NaN in updated weights at iteration {i}\n{updated_weights}")
        # observations = np.array([np.random.choice(observation_replicates) for observation_replicates in replicates]).reshape(-1,)
        # try:
        if CONSTRAINED:
            updated_component_params, updated_weights = (
                constrained_em_iteration(
                    observations,sample_indicators, updated_component_params, updated_weights, xlims, iterNum=i+1,
                )
            )
        else:
            updated_component_params, updated_weights = em_iteration(
                observations,sample_indicators, updated_component_params, updated_weights
            )
        likelihoods = np.array(
            [
                *likelihoods,
                get_likelihood(
                    observations,sample_indicators, updated_component_params, updated_weights
                )
                / len(sample_indicators),
            ]
        )
        if kwargs.get("verbose",True):
            pbar.set_postfix({"likelihood": f"{likelihoods[-1]:.6f}"})
            pbar.update(1)
        if kwargs.get('early_stopping',True) and i >= 1 and (np.abs(likelihoods[-1] - likelihoods[-2]) < 1e-10).all():
            break
    history.append(dict(component_parameters=updated_component_params, weights=updated_weights))
    if kwargs.get("verbose",True):
        pbar.close()
    if CONSTRAINED:
        assert not density_constraint_violated(
            updated_component_params[0], updated_component_params[1], xlims
        )
        
    return dict(component_params=updated_component_params, weights=updated_weights, likelihoods=likelihoods, history=history,kmeans=kmeans)


def fit(observations,sample_indicators,**kwargs):
    fits = []
    LLs = []
    n_fits = kwargs.get("n_fits",1)
    with tqdm(total=n_fits,desc="Fitting",leave=False) as pbar:
        for i in range(n_fits):
            bootstrap_sample_indices = [np.random.choice(np.where(sample_i)[0],
                                                         sample_i.sum(),
                                                         replace=True) for sample_i in sample_indicators.T]
            indices = np.concatenate(bootstrap_sample_indices)
            iter_fit = single_fit(observations[indices],sample_indicators[indices],**kwargs)
            if not len(iter_fit['component_params'][0]):
                continue
            fits.append(iter_fit)
            LLs.append(get_likelihood(observations,
                                      sample_indicators,
                                      iter_fit['component_params'],
                                      iter_fit['weights'])/len(observations))
            llmax = max(LLs)
            pbar.set_description(f"Best LL: {llmax}")
            pbar.update(1)
    
    model_order = np.argsort(LLs)[::-1]
    return fits[model_order[0]]


In [8]:
import scipy.stats as sps
def showFit(X,S,component_params, weights,labels):
    rng = np.linspace(X.min(),X.max(),5000)
    fig,ax = plt.subplots(S.shape[1],1,figsize=(8,3*S.shape[1]),sharex=True,sharey=False)
    f = sps.skewnorm.pdf(rng,component_params[0][0],
                        loc=component_params[0][1],
                        scale=component_params[0][2])
    g = sps.skewnorm.pdf(rng,component_params[1][0],
                    loc=component_params[1][1],
                    scale=component_params[1][2])
    for sampleNum in range(S.shape[1]):
        ax[sampleNum].hist(X[S[:,sampleNum]],bins=15,density=True)
        
        ax[sampleNum].plot(rng,weights[sampleNum,0] * f,linestyle='--',color='red')
        ax[sampleNum].plot(rng,weights[sampleNum,1] * g,linestyle='--',color='blue')
        ax[sampleNum].plot(rng,weights[sampleNum,0] * f+weights[sampleNum,1] *g,alpha=.5,color='green')
        ax[sampleNum].set_title(f"{labels[sampleNum]} (n={S[:,sampleNum].sum()})")
    return fig,ax

In [9]:
data_dir = Path("/Users/danielzeiberg/Desktop/processed_datasets/")
fits = defaultdict(list)
data_files = list(data_dir.glob("*.json"))

In [10]:
savedir = Path('/Users/danielzeiberg/Desktop/testfits')
savedir.mkdir(exist_ok=True,parents=True)
for iteration in trange(10):
    for datafile in data_files:
        dataset_name = datafile.stem.split("_pipeline")[0]
        observations, sample_indicators, labels, bootstrap_indices = prep_data(datafile)
        dataset_fit = fit(observations.ravel(), sample_indicators,
                          Constrained=True,
                          verbose=False,
                          max_iters=10000,
                          n_fits=10,
                          early_stopping=True)
        fits[dataset_name].append(dataset_fit)
        fig,ax = showFit(observations,sample_indicators,
            dataset_fit['component_params'],
            dataset_fit['weights'],labels)
        fig.savefig(savedir / f"{dataset_name}_{iteration+1}.png",bbox_inches='tight',dpi=300)
        plt.close(fig)

  0%|          | 0/10 [00:00<?, ?it/s]

Fitting:   0%|          | 0/10 [00:00<?, ?it/s]

Fitting:   0%|          | 0/10 [00:00<?, ?it/s]

/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_cal

Fitting:   0%|          | 0/10 [00:00<?, ?it/s]

...

/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)


Fitting:   0%|          | 0/10 [00:00<?, ?it/s]

Fitting:   0%|          | 0/10 [00:00<?, ?it/s]

/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_calibration/src/mave_calibration/mave_calibration/skew_normal/density_utils.py:34: RuntimeWarning: divide by zero encountered in log
  numerators = log_pdfs + np.log(individual_sample_weights)
/Users/danielzeiberg/Documents/research/mave_cal

KeyboardInterrupt: 

In [ ]:
dataset_name

In [ ]:
from mave_calibration.skew_normal.density_utils import joint_densities

In [ ]:
# results = [fit(observations,sample_indicators,Constrained=True,verbose=True) for iteration in tqdm(range(100),desc="Iterations",total=100)]
# from joblib import Parallel, delayed
# results = Parallel(n_jobs=100,verbose=100)(delayed(fit)(observations,sample_indicators,Constrained=True,verbose=False) for iteration in range(1000))
result = fit(observations,sample_indicators,
               Constrained=True,
               verbose=True,
               max_iters=10000,
               n_fits=10,
               early_stopping=True)



In [ ]:
result['kmeans'].cluster_centers_

In [ ]:
result['history'][0]['weights']

In [ ]:
showFit(observations,sample_indicators,
        result['history'][0]['component_parameters'],
        result['history'][0]['weights'])

In [ ]:
showFit(observations,sample_indicators,
        result['history'][-1]['component_parameters'],
        result['history'][-1]['weights'])